In [ ]:
!pip install deepcase
from deepcase.preprocessing   import Preprocessor
from deepcase.interpreter import Interpreter
from deepcase.context_builder import ContextBuilder
import numpy as np
import torch
import os
import traceback
from sklearn.metrics import classification_report



In [ ]:
file_path = "hdfs_test_normal.txt"
model_name = "Interpeter"
context_builder = create_model_DEEPCASE(file_path, model_name)

ContextBuilder30ep Esistente


In [ ]:
def create_model_DEEPCASE(file_path, model_name):
    try:
        if os.path.exists("ContextBuilder30ep.save"):
            context_builder = ContextBuilder.load(
                f"ContextBuilder30ep.save"
            )

            print(f"ContextBuilder30ep Esistente")
            return context_builder
        else:
            print(f"Il file {model_name} non esiste. Verra Generato il Modello!")
            preprocessor = Preprocessor(
            length  = 10,    # 10 events in context
            timeout = 86400, # Ignore events older than 1 day (60*60*24 = 86400 seconds)
            )

            context, events, labels, mapping = preprocessor.text(file_path, verbose=True)

            if labels is None:
                labels = np.full(events.shape[0], -1, dtype=int)

            # Split into train and test sets (20:80) by time - assuming events are ordered chronologically
            events_train_n  = events[:events.shape[0]//5 ]
            events_test   = events[ events.shape[0]//5:]

            context_train_n = context[:events.shape[0]//5 ]
            context_test  = context[ events.shape[0]//5:]

            label_train   = labels[:events.shape[0]//5 ]
            label_test    = labels[ events.shape[0]//5:]

            # context_train_n = normalize_tensor(context_train)
            # events_train_n = normalize_tensor(events_train)
            # context_test_n = normalize_tensor(context_test)
            # events_test_n = normalize_tensor(events_test)
            if os.path.exists("ContextBuilder.save"):
              context_builder = ContextBuilder.load(
                  f"ContextBuilder.save"
              )
            else:

              # Create ContextBuilder
              context_builder = ContextBuilder(
                  input_size    = 100,   # Number of input features to expect
                  output_size   = 100,   # Same as input size
                  hidden_size   = 128,   # Number of nodes in hidden layer, in paper we set this to 128
                  max_length    = 10,    # Length of the context, should be same as context in Preprocessor
              )

              context_builder.fit(
                  X             = context_train_n,               # Context to train with
                  y             = events_train_n.reshape(-1, 1), # Events to train with, note that these should be of shape=(n_events, 1)
                  epochs        = 30,                          # Number of epochs to train with
                  batch_size    = 128,                         # Number of samples in each training batch, in paper this was 128
                  learning_rate = 0.01,                        # Learning rate to train with, in paper this was 0.01
                  verbose       = True,                        # If True, prints progress
              )

              context_builder.save('ContextBuilder.save')

            ########################################################################
            #                  Get prediction from ContextBuilder                  #
            ########################################################################

            # Use context builder to predict confidence
            confidence, _ = context_builder.predict(
                X = context_test
            )

            # Get confidence of the next step, seq_len 0 (n_samples, seq_len, output_size)
            confidence = confidence[:, 0]
            # Get confidence from log confidence
            confidence = confidence.exp()
            # Get prediction as maximum confidence
            y_pred = confidence.argmax(dim=1)

            ########################################################################
            #                          Perform evaluation                          #
            ########################################################################

            # Get test and prediction as numpy array
            y_test = events_test.cpu().numpy()
            y_pred = y_pred     .cpu().numpy()

            # Print classification report
            print(classification_report(
                y_true = y_test,
                y_pred = y_pred,
                digits = 4,
            ))

            # # Create Interpreter
            # interpreter = Interpreter(
            #     context_builder = context_builder, # ContextBuilder used to fit data
            #     features        = 100,             # Number of input features to expect, should be same as ContextBuilder
            #     eps             = 0.1,             # Epsilon value to use for DBSCAN clustering, in paper this was 0.1
            #     min_samples     = 5,               # Minimum number of samples to use for DBSCAN clustering, in paper this was 5
            #     threshold       = 0.2,             # Confidence threshold used for determining if attention from the ContextBuilder can be used, in paper this was 0.2
            # )

            # # Cluster samples with the interpreter
            # clusters = interpreter.cluster(
            #     X          = context_train_n,               # Context to train with
            #     y          = events_train_n.reshape(-1, 1), # Events to train with, note that these should be of shape=(n_events, 1)
            #     iterations = 100,                         # Number of iterations to use for attention query, in paper this was 100
            #     batch_size = 1024,                        # Batch size to use for attention query, used to limit CUDA memory usage
            #     verbose    = True,                        # If True, prints progress
            # )


            # # Compute scores for each cluster based on individual labels per sequence
            # scores = interpreter.score_clusters(
            #     scores   = label_train, # Labels used to compute score (either as loaded by Preprocessor, or put your own labels here)
            #     strategy = "max",        # Strategy to use for scoring (one of "max", "min", "avg")
            #     NO_SCORE = -1,           # Any sequence with this score will be ignored in the strategy.
            #                             # If assigned a cluster, the sequence will inherit the cluster score.
            #                             # If the sequence is not present in a cluster, it will receive a score of NO_SCORE.
            # )

            # # Assign scores to clusters in interpreter
            # # Note that all sequences should be given a score and each sequence in the
            # # same cluster should have the same score.
            # interpreter.score(
            #     scores  = scores, # Scores to assign to sequences
            #     verbose = True,   # If True, prints progress
            # )

            # interpreter.save(f"{model_name}")
            # # evalute_model_DEEPCASE(interpreter, file_path)
            return True

    except Exception as e:
        # Gestisci altre eccezioni se si verificano
        print(f"Si è verificato un errore: {e}")
        traceback.print_exc()


        return None

In [ ]:
preprocessor = Preprocessor(length  = 10,timeout = 86400)
context, events, labels, mapping = preprocessor.text(file_path, verbose=True)

if labels is None:
    labels = np.full(events.shape[0], -1, dtype=int)

# Split into train and test sets (20:80) by time - assuming events are ordered chronologically
events_train_n  = events[:events.shape[0]//5 ]
events_test   = events[ events.shape[0]//5:]

context_train_n = context[:events.shape[0]//5 ]
context_test  = context[ events.shape[0]//5:]

label_train   = labels[:events.shape[0]//5 ]
label_test    = labels[ events.shape[0]//5:]
 # Use context builder to predict confidence
confidence, _ = context_builder.predict(
    X = context_test
)

# Get confidence of the next step, seq_len 0 (n_samples, seq_len, output_size)
confidence = confidence[:, 0]
# Get confidence from log confidence
confidence = confidence.exp()
# Get prediction as maximum confidence
y_pred = confidence.argmax(dim=1)

########################################################################
#                          Perform evaluation                          #
########################################################################

# Get test and prediction as numpy array
y_test = events_test.cpu().numpy()
y_pred = y_pred     .cpu().numpy()

# Print classification report
print(classification_report(
    y_true = y_test,
    y_pred = y_pred,
    digits = 4,
))

Loading: 100%|██████████| 302984/302984 [01:26<00:00, 3507.13it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0     0.6090    0.1832    0.2817     50178
           1     0.8451    0.5529    0.6685    178358
           2     0.5898    0.8079    0.6818    149575
           3     0.8195    0.9997    0.9006    729051
           4     0.6411    0.9057    0.7507      1124
           5     0.9950    0.9873    0.9911    727926
           6     0.9106    0.9366    0.9234    727926
           7     0.8625    0.4791    0.6160      1100
           8     0.9628    0.8474    0.9014      1101
           9     0.0000    0.0000    0.0000        97
          10     0.9990    0.9997    0.9993    593633
          11     0.9994    0.3402    0.5077    242643
          12     0.8905    0.9529    0.9207    592612
          13     0.7429    0.3333    0.4602      1101
          14     0.9382    0.9188    0.9284    730014
          15     0.0000    0.0000    0.0000         2
          16     0.0000    0.0000    0.0000         5

    accuracy              

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def normalize_tensor(tensor_data):
  tensor_min = torch.min(tensor_data)
  tensor_max = torch.max(tensor_data)

  # Normalizza il Tensor nell'intervallo [0, 1]
  normalized_tensor = (tensor_data - tensor_min) / (tensor_max - tensor_min)

  # Puoi anche normalizzare in un intervallo specifico, ad esempio [0, 99]
  new_min = 0
  new_max = 99
  normalized_tensor_specific_range = \
   (tensor_data - tensor_min) / (tensor_max - tensor_min) \
        * (new_max - new_min) + new_min

  return normalized_tensor_specific_range.to(torch.long)

